Loaded pretrained weights for efficientnet-b0


RuntimeError: Numpy is not available

In [1]:
import pandas as pd
import os
import numpy as np
import json
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
import matplotlib.pyplot as plt
import time
import copy
import random


In [9]:
#Create Files_Name
image_data='C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
pd.DataFrame(os.listdir(image_data),columns=['Files_Name'])

,Files_Name
0,Bird-drop
1,Clean
2,Dusty
3,Electrical-damage
4,Physical-Damage
5,Snow-Covered


In [ ]:
img_height = 244
img_width = 244
train_ds = tf.keras.utils.image_dataset_from_directory(
  'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel',
  validation_split=0.2,
  subset='training',
  image_size=(img_height, img_width),
  batch_size=32,
  seed=42,
  shuffle=True)

val_ds = tf.keras.utils.image_dataset_from_directory(
  'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel',
  validation_split=0.2,
  subset='validation',
  image_size=(img_height, img_width),
  batch_size=32,
  seed=42,
  shuffle=True)

In [15]:

from efficientnet_pytorch import EfficientNet
model_name = 'efficientnet-b0'  # b5

image_size = EfficientNet.get_image_size(model_name)
print(image_size)
model = EfficientNet.from_pretrained(model_name, num_classes=6)

224
Loaded pretrained weights for efficientnet-b0


In [1]:
import pandas as pd
import os
import numpy as np
import json
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
import matplotlib.pyplot as plt
import time
import copy
import random


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

# 하이퍼파라미터 설정
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 128
epochs = 20
learning_rate = 0.001
image_size = EfficientNet.get_image_size(model_name)

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드 및 분리
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# 데이터셋 분할 (80% 훈련, 20% 검증)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)
print(device)

Loaded pretrained weights for efficientnet-b0
cuda


In [ ]:
# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 함수
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 통계
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 평가 함수
def test(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 통계
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 학습 및 검증 루프
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

print("Training complete!")


Epoch 1/20
Train Loss: 1.3880, Train Accuracy: 0.5607
Val Loss: 2.1020, Val Accuracy: 0.2938
Epoch 2/20
Train Loss: 0.4531, Train Accuracy: 0.8927
Val Loss: 1.9923, Val Accuracy: 0.5311
Epoch 3/20
Train Loss: 0.1540, Train Accuracy: 0.9605
Val Loss: 2.4778, Val Accuracy: 0.5763
Epoch 4/20
Train Loss: 0.0751, Train Accuracy: 0.9816
Val Loss: 2.9358, Val Accuracy: 0.6045
Epoch 5/20
Train Loss: 0.0560, Train Accuracy: 0.9929
Val Loss: 2.3301, Val Accuracy: 0.6441
Epoch 6/20
Train Loss: 0.0309, Train Accuracy: 0.9915
Val Loss: 1.4807, Val Accuracy: 0.7119
Epoch 7/20
Train Loss: 0.0233, Train Accuracy: 0.9944
Val Loss: 1.3553, Val Accuracy: 0.7627
Epoch 8/20
Train Loss: 0.0239, Train Accuracy: 0.9929
Val Loss: 1.2535, Val Accuracy: 0.7627
Epoch 9/20
Train Loss: 0.0314, Train Accuracy: 0.9901
Val Loss: 1.1238, Val Accuracy: 0.8192
Epoch 10/20
Train Loss: 0.0260, Train Accuracy: 0.9958
Val Loss: 1.0672, Val Accuracy: 0.8079
Epoch 11/20
Train Loss: 0.0170, Train Accuracy: 0.9958
Val Loss: 1.27